# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )
root = document_tree.getroot()
print(root)

<Element 'mondial' at 0x000001F5DDBD7548>


In [5]:
# print names of all countries
for child in root:  # find root element
    print(child.find('name').text) # print(child.tag)  find a specific single element in the document.

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [6]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    # Finds all matching subelements, by tag name or path. Returns an iterable
    # yielding all matching elements in document order.
    print('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.iter('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella



****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [8]:
document = ET.parse( './data/mondial_database.xml' )

In [9]:
# 1. output 10 countries with the lowest infant mortality rates
country_mortality = []
for element in document.iterfind('country'): 
    temp = element.find('infant_mortality') # .find() returns a node (Element)
    if temp is not None:
        country_mortality.append((element.find('name').text, float(temp.text)))

country_mortality.sort(key=lambda x: x[1], reverse = True)
print(country_mortality[0:10])

[('Western Sahara', 145.82), ('Afghanistan', 117.23), ('Mali', 104.34), ('Somalia', 100.14), ('Central African Republic', 92.86), ('Guinea-Bissau', 90.92), ('Chad', 90.3), ('Niger', 86.27), ('Angola', 79.99), ('Burkina Faso', 76.8)]


In [12]:
# 1. 10 cities with the largest population listed in the xml file
city_population = []
for element in document.iterfind('country'):
    for subelement in element.iter('city'):
        pops = subelement.findall('population') 
        if pops:
            city_population.append((subelement.find('name').text, int(pops[-1].text))) # use largest population on file
                                   
city_population.sort(key=lambda x: x[1], reverse = True)
print(city_population[0:10])


[('Shanghai', 22315474), ('Istanbul', 13710512), ('Mumbai', 12442373), ('Moskva', 11979529), ('Beijing', 11716620), ('São Paulo', 11152344), ('Tianjin', 11090314), ('Guangzhou', 11071424), ('Delhi', 11034555), ('Shenzhen', 10358381)]


In [13]:
# 2. 10 cities with the largest population in 2011, those without data is excluded.
city_population = []
for element in document.iterfind('country'):
    for subelement in element.iter('city'):
        pops = subelement.find("population[@year='2011']")
        # .findall() returns a list of nodes, or an empty list of none found
        if pops is not None:
            city_population.append((subelement.find('name').text, int(pops.text)))
                                   
city_population.sort(key=lambda x: x[1], reverse = True)
print(city_population[0:10])


[('Mumbai', 12442373), ('Delhi', 11034555), ('Bangalore', 8443675), ('London', 8250205), ('Tehran', 8154051), ('Dhaka', 7423137), ('Hyderabad', 6731790), ('Ahmadabad', 5577940), ('Luanda', 5000000), ('Chennai', 4646732)]


In [14]:
# 3. 10 ethnic groups with the largest overall populations 
# (sum of best/latest estimates over all countries)
ethinc_population = []
for element in document.iter('country'):
    pops = element.findall('ethnicgroup') 
    temp = element.findall('population') 
    # only calculating those with both country population and ethnicgroup
    if pops and temp:
        for i in range(len(pops)):
            # only focus on the last population
            ethinc_population.append((pops[i].text, int(int(temp[-1].text)*float(pops[i].get('percentage'))/100)))                                                    

ethinc_population.sort(key=lambda x: x[1], reverse = True)
print(ethinc_population[0:10])

[('Han Chinese', 1245058800), ('Indo-Aryan', 871815583), ('Dravidian', 302713744), ('European', 254958101), ('African', 162651570), ('Bengali', 146776916), ('Japanese', 126534212), ('Russian', 114646210), ('Javanese', 113456006), ('European', 108886717)]
